In [ ]:
import glob
import math
import uproot
import numpy as np
import pandas as pd
import pkgutil
import uproot_methods

import functions_giammi as fugi
from importlib import reload

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
%matplotlib inline
%config InlineBackend.figure_formats=["png"]

import re

import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import itertools
from scipy.ndimage.filters import gaussian_filter

from __future__ import division

plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

# np.warnings.filterwarnings('error', category=np.VisibleDeprecationWarning)

#loads the temperaure cmap like Philipp for both matpplotlib and plotly
cmap_temp, cmap_temp_go, Magma_r, Seismic_r = fugi.customcmaps()

#to see which classes have been defined so far
[modname for importer, modname, ispkg in pkgutil.walk_packages(uproot_methods.classes.__path__)]

# fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R_MetOx_33eV_RCP_m30-28_new.root")
# fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R_MetOx_33eV_LCP_m30-28_new.root")
# fileSCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S_MetOx_33eV_RCP_m30-28_new.root")
# fileSCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S_MetOx_33eV_LCP_m30-28_new.root")

fileRCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R_MetOx_33eV_RCP_m30-28_new_all.root")
fileRCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/R_MetOx_33eV_LCP_m30-28_new_all.root")
fileSCR = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S_MetOx_33eV_RCP_m30-28_new_all.root")
fileSCL = uproot.open(r"../PYTHON_graphs/DATA/Experiments/S_MetOx_33eV_LCP_m30-28_new_all.root")

DIR = ["if_twoelec/","if_e[0]_valid/"]
EN = ["NO_gate","30%_en_e1","50%_en_e1","80%_en_e1","e12_sym/NO_gate","e12_sym/50%_sh_en","e12_sym/80%_sh_en"]

# EN_opt = 0; ID_P=5; ID_C=5; ID_M=5
EN_opt = 0; ID_P=2; ID_C=2; ID_M=2 #to be used just with e12_sym series 4, 5, 6!

# the next layer is L or R, selected later
ID_MFPAD=["MFPAD_e1_MF","MFPAD_e2_MF","MFPAD_esum_MF","MFPAD_e1_MF_phi","MFPAD_e2_MF_phi","MFPAD_esum_MF_phi", "MFPAD_sym_MF"]
ID_PECD=["PECD_e[e1](theta)","PECD_e[e2](theta)","PECD_pesum(theta)","PECD_e[e1](theta)_phi","PECD_e[e2](theta)_phi","PECD_pesum(theta)_phi", "PECD_sym(theta)", "PECD_sym(theta)_phi"]
ID_COS=["cos(theta)_e[e1]","cos(theta)_e[e2]","cos(theta)_pesum","cos(theta)_e[e1]_phi","cos(theta)_e[e2]_phi","cos(theta)_pesum_phi","cos(theta)_sym", "cos(theta)_sym_phi"]

#twoelec valid
loc_two="angular_distr_el/"+DIR[0]+EN[EN_opt]+"/"
#just e[0]
loc_e0="angular_distr_el/"+DIR[1]+EN[EN_opt]+"/"
loc_tot="angular_distr_el/all_nogate/"

save_path="../PYTHON_graphs/OUTPUTS/doubleionization/"

if len([el for el in str(EN[EN_opt]).split("_") if "%" in el]) > 0 and EN_opt<4:
    if len([el for el in str(ID_PECD[ID_P]).split("_") if "phi" in el]) > 0:
        title = str(EN[EN_opt]).split("_")[0]+"_"+str(ID_PECD[ID_P]).split("_")[1].split("(")[0] + "_reduced"
    else:
        title = str(EN[EN_opt]).split("_")[0]+"_"+str(ID_PECD[ID_P]).split("_")[1].split("(")[0]
else:
    if len([el for el in str(ID_PECD[ID_P]).split("_") if "phi" in el]) > 0:
        title = str(ID_PECD[ID_P]).split("_")[1].split("(")[0] + "_reduced"
    else:
        title = str(ID_PECD[ID_P]).split("_")[1].split("(")[0]

print(title)

In [ ]:
#commands for uproot single graphs loading
fileRCR[loc_two+ID_PECD[ID_P]].all_members
fileRCR[loc_two+ID_PECD[ID_P]].axis("x").centers() #N
fileRCR[loc_two+ID_PECD[ID_P]].to_numpy()

In [ ]:
cos_RCR_all = fileRCR[loc_tot+"cos(theta)_e[0]"].to_numpy()[0]
cos_RCL_all = fileRCL[loc_tot+"cos(theta)_e[0]"].to_numpy()[0]
cos_SCR_all = fileSCR[loc_tot+"cos(theta)_e[0]"].to_numpy()[0]
cos_SCL_all = fileSCL[loc_tot+"cos(theta)_e[0]"].to_numpy()[0]

# cos_RCR_all_el = fileRCR[loc_tot+"cos(theta)_e[0]_el_red"].to_numpy()[0]
# cos_RCL_all_el = fileRCL[loc_tot+"cos(theta)_e[0]_el_red"].to_numpy()[0]
# cos_SCR_all_el = fileSCR[loc_tot+"cos(theta)_e[0]_el_red"].to_numpy()[0]
# cos_SCL_all_el = fileSCL[loc_tot+"cos(theta)_e[0]_el_red"].to_numpy()[0]

cos_RCR_all_el = fileRCR[loc_tot+"cos(theta)_e[0]"].to_numpy()[0]
cos_RCL_all_el = fileRCL[loc_tot+"cos(theta)_e[0]"].to_numpy()[0]
cos_SCR_all_el = fileSCR[loc_tot+"cos(theta)_e[0]"].to_numpy()[0]
cos_SCL_all_el = fileSCL[loc_tot+"cos(theta)_e[0]"].to_numpy()[0]

cos_err_all = fileSCL[loc_tot+"cos(theta)_e[0]"].errors()

PECD_RCR = fileRCR[loc_two+ID_PECD[ID_P]].to_numpy()[0]
MFPAD_RCR = fileRCR[loc_two+ID_MFPAD[ID_M]].to_numpy()[0]
cos_RCR = fileRCR[loc_two+ID_COS[ID_C]].to_numpy()[0]
cos_RCR_err = fileRCR[loc_two+ID_COS[ID_C]].errors()

PECD_RCL = fileRCL[loc_two+ID_PECD[ID_P]].to_numpy()[0]
MFPAD_RCL = fileRCL[loc_two+ID_MFPAD[ID_M]].to_numpy()[0]
cos_RCL = fileRCL[loc_two+ID_COS[ID_C]].to_numpy()[0]
cos_RCL_err = fileRCL[loc_two+ID_COS[ID_C]].errors()

PECD_SCR = fileSCR[loc_two+ID_PECD[ID_P]].to_numpy()[0]
MFPAD_SCR = fileSCR[loc_two+ID_MFPAD[ID_M]].to_numpy()[0]
cos_SCR = fileSCR[loc_two+ID_COS[ID_C]].to_numpy()[0]
cos_SCR_err = fileSCR[loc_two+ID_COS[ID_C]].errors()

PECD_SCL = fileSCL[loc_two+ID_PECD[ID_P]].to_numpy()[0]
MFPAD_SCL = fileSCL[loc_two+ID_MFPAD[ID_M]].to_numpy()[0]
cos_SCL = fileSCL[loc_two+ID_COS[ID_C]].to_numpy()[0]
cos_SCL_err = fileSCL[loc_two+ID_COS[ID_C]].errors()

phiM = fileRCR[loc_two+ID_MFPAD[ID_M]].axis("x").centers()
cthetaM = fileRCR[loc_two+ID_MFPAD[ID_M]].axis("y").centers()
cthetaP = fileRCR[loc_two+ID_PECD[ID_P]].axis("y").centers()
ctheta = fileRCR[loc_two+ID_COS[ID_C]].axis("x").centers()
ctheta_all = fileRCR[loc_tot+"cos(theta)_e[0]_el"].axis("x").centers()
beta = fileRCR[loc_two+ID_PECD[ID_P]].axis("x").centers()

print("phiM shape = ", phiM.shape)
print("cthetaM shape = ", cthetaM.shape)
print("cthetaP shape = ", cthetaP.shape)
print("ctheta shape = ", ctheta.shape)
print("beta shape = ", beta.shape)
print("PECD shape = ", PECD_RCR.shape)
print("MFPAD shape = ", MFPAD_RCR.shape)

PECD_RCR_norm = fugi.normalise_with_err(PECD_RCR)
MFPAD_RCR_norm = fugi.normalise_with_err(MFPAD_RCR)
cos_RCR_norm = fugi.normalise_with_err(cos_RCR)
cos_RCR_all_norm = fugi.normalise_with_err(cos_RCR_all)

PECD_RCL_norm = fugi.normalise_with_err(PECD_RCL)
MFPAD_RCL_norm = fugi.normalise_with_err(MFPAD_RCL)
cos_RCL_norm = fugi.normalise_with_err(cos_RCL)
cos_RCL_all_norm = fugi.normalise_with_err(cos_RCL_all)

PECD_SCR_norm = fugi.normalise_with_err(PECD_SCR)
MFPAD_SCR_norm = fugi.normalise_with_err(MFPAD_SCR)
cos_SCR_norm = fugi.normalise_with_err(cos_SCR)
cos_SCR_all_norm = fugi.normalise_with_err(cos_SCR_all)

PECD_SCL_norm = fugi.normalise_with_err(PECD_SCL)
MFPAD_SCL_norm = fugi.normalise_with_err(MFPAD_SCL)
cos_SCL_norm = fugi.normalise_with_err(cos_SCL)
cos_SCL_all_norm = fugi.normalise_with_err(cos_SCL_all)


#cos: SHOULD BE FLIPPED, BUT THE IDEA IS THAT THE HELICITY HAS NOT BEEN REVERSED DURING THE EXPERIMENT
cos_S = cos_SCR+cos_SCL #np.flip(cos_SCL)    
cos_R = cos_RCR+cos_RCL #np.flip(cos_RCL)   
cos_S_norm = fugi.normalise_with_err(cos_S)
cos_R_norm = fugi.normalise_with_err(cos_R)

cPECD_tot_R = np.divide(np.subtract(cos_RCR_norm,cos_RCL_norm),np.add(cos_RCR_norm,cos_RCL_norm))
cPECD_tot_R_err = fugi.error_calc(cos_RCR,cos_RCL)

cPECD_tot_S = np.divide(np.subtract(cos_SCR_norm,cos_SCL_norm),np.add(cos_SCR_norm,cos_SCL_norm))
cPECD_tot_S_err = fugi.error_calc(cos_SCR,cos_SCL)

cPECD_tot_RS = np.divide(np.subtract(cos_R_norm,cos_S_norm),np.add(cos_R_norm,cos_S_norm))
cPECD_tot_RS_err = fugi.error_calc(cos_R,cos_S)

cPECD_all_R = np.divide(np.subtract(cos_RCR_all_norm,cos_RCL_all_norm),np.add(cos_RCR_all_norm,cos_RCL_all_norm))
cPECD_all_R_err = fugi.error_calc(cos_RCR_all,cos_RCL_all)

cPECD_all_S = np.divide(np.subtract(cos_SCR_all_norm,cos_SCL_all_norm),np.add(cos_SCR_all_norm,cos_SCL_all_norm))
cPECD_all_S_err = fugi.error_calc(cos_SCR_all,cos_SCL_all)


#PECD maps: SHOULD BE FLIPPED, BUT THE IDEA IS THAT THE HELICITY HAS NOT BEEN REVERSED DURING THE EXPERIMENT
PECD_S = PECD_SCR+PECD_SCL #np.flip(PECD_SCL,axis=0) 
PECD_R = PECD_RCR+PECD_RCL #np.flip(PECD_RCL,axis=0)

PECD_S_norm = fugi.normalise_with_err(PECD_S)
PECD_R_norm = fugi.normalise_with_err(PECD_R)

#set a sensible gaussian value
gaussian=0.6

PECD_tot_RS = np.divide(np.subtract(PECD_R_norm,PECD_S_norm),np.add(PECD_R_norm,PECD_S_norm))
PECD_tot_RS_presmooth = np.divide(np.subtract(gaussian_filter(PECD_R_norm,sigma=gaussian),gaussian_filter(PECD_S_norm,sigma=gaussian)),np.add(gaussian_filter(PECD_R_norm,sigma=gaussian),gaussian_filter(PECD_S_norm,sigma=gaussian)))
# PECD_tot = np.divide(np.subtract(PECD_S,PECD_R),np.add(PECD_S,PECD_R))

PECD_tot_R = np.divide(np.subtract(PECD_RCR_norm,PECD_RCL_norm),np.add(PECD_RCR_norm,PECD_RCL_norm))
PECD_tot_S = np.divide(np.subtract(PECD_SCR_norm,PECD_SCL_norm),np.add(PECD_SCR_norm,PECD_SCL_norm))

#MFPAD
MFPAD_S = MFPAD_SCR+MFPAD_SCL #np.flip(MFPAD_SCL,axis=0) 
MFPAD_R = MFPAD_RCR+MFPAD_RCL #np.flip(MFPAD_RCL,axis=0)

MFPAD_S_norm = fugi.normalise_with_err(MFPAD_S)
MFPAD_R_norm = fugi.normalise_with_err(MFPAD_R)

MFPAD_tot_RS = np.divide(np.subtract(MFPAD_R_norm,MFPAD_S_norm),np.add(MFPAD_R_norm,MFPAD_S_norm))
MFPAD_tot_RS_presmooth = np.divide(np.subtract(gaussian_filter(MFPAD_R_norm,sigma=gaussian),gaussian_filter(MFPAD_S_norm,sigma=gaussian)),np.add(gaussian_filter(MFPAD_R_norm,sigma=gaussian),gaussian_filter(MFPAD_S_norm,sigma=gaussian)))

MFPAD_tot_R = np.divide(np.subtract(MFPAD_RCR_norm,MFPAD_RCL_norm),np.add(MFPAD_RCR_norm,MFPAD_RCL_norm))
MFPAD_tot_S = np.divide(np.subtract(MFPAD_SCR_norm,MFPAD_SCL_norm),np.add(MFPAD_SCR_norm,MFPAD_SCL_norm))

In [ ]:
#INTESNITY OF THE COS(THETA)
#SINGLE ELICITIES
import functions_minuit as fumi

fig, (ax1,ax2) = plt.subplots(1,2, figsize=(20,10))
custom_xlim = (-1, 1)
    
limits=[(None, None), (-1, 1), (-1, 2)]

x_data = ctheta;y_data = cos_RCR;y_err = cos_RCR_err

m1=fumi.iminuit_err_cos(x_data,y_data,y_err,limits)
fumi.plotminuit_cos(x_data,y_data,y_err,m1,ax1,custom_xlim)
ax1.set_title(title+" R CR 33 eV")
ax1.set_xlabel('cos(\u03D1) [adm]')
ax1.set_ylabel('counts')

fit_info = []
for p, v, e in zip(m1.parameters, m1.values, m1.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

ax1.legend(title="\n".join(fit_info), loc="best")

# aspect=(custom_xlim[1]/custom_ylim[1])
# ax1.set_aspect(aspect)

print("CR fitting and covarience \n", m1, "\n")

x_data = ctheta;y_data = cos_RCL;y_err = cos_RCL_err

m2=fumi.iminuit_err_cos(x_data,y_data,y_err,limits)
fumi.plotminuit_cos(x_data,y_data,y_err,m2,ax2,custom_xlim)
ax2.set_title(title+" R CL 33 eV")
ax2.set_xlabel('cos(\u03D1) [adm]')
ax2.set_ylabel('counts')

fit_info = []
for p, v, e in zip(m2.parameters, m2.values, m2.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

ax2.legend(title="\n".join(fit_info), loc="best")

# aspect=(custom_xlim[1]/custom_ylim[1])
# ax2.set_aspect(aspect)


print("CL fitting and covarience \n", m2)

fig.savefig(save_path+title+"_intesity_R-CR-CL", dpi=144, transparent=False, bbox_inches='tight')

In [ ]:
#INTESNITY OF THE COS(THETA)
#SINGLE ELICITIES
import functions_minuit as fumi

fig, (ax1,ax2) = plt.subplots(1,2, figsize=(20,10))
custom_xlim = (-1, 1)
    
limits=[(None, None), (-1, 1), (-1, 2)]

x_data = ctheta;y_data = cos_SCR;y_err = cos_SCR_err

m1=fumi.iminuit_err_cos(x_data,y_data,y_err,limits)
fumi.plotminuit_cos(x_data,y_data,y_err,m1,ax1,custom_xlim)
ax1.set_title(title+" S CR 33 eV")
ax1.set_xlabel('cos(\u03D1) [adm]')
ax1.set_ylabel('counts')

fit_info = []
for p, v, e in zip(m1.parameters, m1.values, m1.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

ax1.legend(title="\n".join(fit_info), loc="best")

# aspect=(custom_xlim[1]/custom_ylim[1])
# ax1.set_aspect(aspect)

print("CR fitting and covarience \n", m1, "\n")

x_data = ctheta;y_data = cos_SCL;y_err = cos_SCL_err

m2=fumi.iminuit_err_cos(x_data,y_data,y_err,limits)
fumi.plotminuit_cos(x_data,y_data,y_err,m2,ax2,custom_xlim)
ax2.set_title(title+" S CL 33 eV")
ax2.set_xlabel('cos(\u03D1) [adm]')
ax2.set_ylabel('counts')

fit_info = []
for p, v, e in zip(m2.parameters, m2.values, m2.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

ax2.legend(title="\n".join(fit_info), loc="best")

# aspect=(custom_xlim[1]/custom_ylim[1])
# ax2.set_aspect(aspect)


print("CL fitting and covarience \n", m2)

fig.savefig(save_path+title+"_intesity_S-CR-CL", dpi=144, transparent=False, bbox_inches='tight')

In [ ]:
import functions_minuit as fumi
#PECD INTERPOLATION RANDOM
# import functions_minuit as fumi

fig, ax = plt.subplots()
custom_xlim = (-1, 1)
custom_ylim = (-0.05, 0.05)
    
# limits=[(-1, 1), (-1, 1),(-1, 1), (-1, 1),(-1, 1), (-1, 1)] #PECD6
limits=[(-1, 1), (-1, 2)] #PECD2

x_data = ctheta; y_data = cPECD_tot_R; y_err = cPECD_tot_R_err

m=fumi.iminuit_err2(x_data,y_data,y_err,limits)
fumi.plotminuit2(x_data,y_data,y_err,m,ax,custom_xlim, ylim=custom_ylim)

ax.set_title(title+" int PECD R CR-CL @ 33 eV")

ax.set_xlabel('cos(\u03D1) [adm]')
ax.set_ylabel('PECD')

fit_info = []
for p, v, e in zip(m.parameters, m.values, m.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

ax.legend(title="\n".join(fit_info), loc="best")

aspect=(custom_xlim[1]/custom_ylim[1])
ax.set_aspect(aspect)

m

fig.savefig(save_path+title+"_intPECD_R-CR-CL", dpi=144, transparent=False, bbox_inches='tight')

In [ ]:
import functions_minuit as fumi
#PECD INTERPOLATION RANDOM
# import functions_minuit as fumi

fig, ax = plt.subplots()
custom_xlim = (-1, 1)
custom_ylim = (-0.05, 0.05)
    
# limits=[(-1, 1), (-1, 1),(-1, 1), (-1, 1),(-1, 1), (-1, 1)] #PECD6
limits=[(-1, 1), (-1, 2)] #PECD2

x_data = ctheta; y_data = cPECD_tot_S; y_err = cPECD_tot_S_err

m=fumi.iminuit_err2(x_data,y_data,y_err,limits)
fumi.plotminuit2(x_data,y_data,y_err,m,ax,custom_xlim, ylim=custom_ylim)

ax.set_title(title+" int PECD S CR-CL @ 33 eV")

ax.set_xlabel('cos(\u03D1) [adm]')
ax.set_ylabel('PECD')

fit_info = []
for p, v, e in zip(m.parameters, m.values, m.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

ax.legend(title="\n".join(fit_info), loc="best")

aspect=(custom_xlim[1]/custom_ylim[1])
ax.set_aspect(aspect)

m

fig.savefig(save_path+title+"_PECD_S-CR-CL", dpi=144, transparent=False, bbox_inches='tight')

In [ ]:
import functions_minuit as fumi
#PECD INTERPOLATION RANDOM
# import functions_minuit as fumi

fig, ax = plt.subplots()
custom_xlim = (-1, 1)
custom_ylim = (-0.05, 0.05)
    
# limits=[(-1, 1), (-1, 1),(-1, 1), (-1, 1),(-1, 1), (-1, 1)] #PECD6
limits=[(-1, 1), (-1, 2)] #PECD2

x_data = ctheta;y_data = cPECD_tot_RS;y_err = cPECD_tot_RS_err

m=fumi.iminuit_err2(x_data,y_data,y_err,limits)
fumi.plotminuit2(x_data,y_data,y_err,m,ax,custom_xlim, ylim=custom_ylim)

ax.set_title(title+" int PECD R-S @ 33 eV")

ax.set_xlabel('cos(\u03D1) [adm]')
ax.set_ylabel('PECD')

fit_info = []
for p, v, e in zip(m.parameters, m.values, m.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

ax.legend(title="\n".join(fit_info), loc="best")

aspect=(custom_xlim[1]/custom_ylim[1])
ax.set_aspect(aspect)

m

fig.savefig(save_path+title+"_intPECD_R-S", dpi=144, transparent=False, bbox_inches='tight')

In [ ]:
#INTESNITY OF THE COS(THETA)
#SINGLE ELICITIES
import functions_minuit as fumi

fig, (ax1,ax2) = plt.subplots(1,2, figsize=(20,10))
custom_xlim = (-1, 1)
    
limits=[(None, None), (-1, 1), (-1, 2)]

x_data = ctheta_all;y_data = cos_RCR_all_el;y_err = cos_err_all

m1=fumi.iminuit_err_cos(x_data,y_data,y_err,limits)
fumi.plotminuit_cos(x_data,y_data,y_err,m1,ax1,custom_xlim)
ax1.set_title("R CR all 33 eV")
ax1.set_xlabel('cos(\u03D1) [adm]')
ax1.ticklabel_format(axis="y",style="sci",scilimits=(0,1))
ax1.set_ylabel('counts')

fit_info = []
for p, v, e in zip(m1.parameters, m1.values, m1.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

ax1.legend(title="\n".join(fit_info), loc="best")

# aspect=(custom_xlim[1]/custom_ylim[1])
# ax1.set_aspect(aspect)

print("CR fitting and covarience \n", m1, "\n")

x_data = ctheta_all;y_data = cos_RCL_all_el;y_err = cos_err_all

m2=fumi.iminuit_err_cos(x_data,y_data,y_err,limits)
fumi.plotminuit_cos(x_data,y_data,y_err,m2,ax2,custom_xlim)
ax2.set_title(" R CL all 33 eV")
ax2.set_xlabel('cos(\u03D1) [adm]')
ax2.ticklabel_format(axis="y",style="sci",scilimits=(0,1))
ax2.set_ylabel('counts')

fit_info = []
for p, v, e in zip(m2.parameters, m2.values, m2.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

ax2.legend(title="\n".join(fit_info), loc="best")

# aspect=(custom_xlim[1]/custom_ylim[1])
# ax2.set_aspect(aspect)


print("CL fitting and covarience \n", m2)

# fig.savefig(save_path+title+"_intesity_R-CR-CL", dpi=144, transparent=False, bbox_inches='tight')

In [ ]:
#INTESNITY OF THE COS(THETA)
#SINGLE ELICITIES
import functions_minuit as fumi

fig, (ax1,ax2) = plt.subplots(1,2, figsize=(20,10))
custom_xlim = (-1, 1)
    
limits=[(None, None), (-1, 1), (-1, 2)]

x_data = ctheta_all;y_data = cos_SCR_all_el;y_err = cos_err_all

m1=fumi.iminuit_err_cos(x_data,y_data,y_err,limits)
fumi.plotminuit_cos(x_data,y_data,y_err,m1,ax1,custom_xlim)
ax1.set_title("S CR all 33 eV")
ax1.set_xlabel('cos(\u03D1) [adm]')
ax1.ticklabel_format(axis="y",style="sci",scilimits=(0,1))
ax1.set_ylabel('counts')

fit_info = []
for p, v, e in zip(m1.parameters, m1.values, m1.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

ax1.legend(title="\n".join(fit_info), loc="best")

# aspect=(custom_xlim[1]/custom_ylim[1])
# ax1.set_aspect(aspect)

print("CR fitting and covarience \n", m1, "\n")

x_data = ctheta_all;y_data = cos_SCL_all_el;y_err = cos_err_all

m2=fumi.iminuit_err_cos(x_data,y_data,y_err,limits)
fumi.plotminuit_cos(x_data,y_data,y_err,m2,ax2,custom_xlim)
ax2.set_title(" S CL all 33 eV")
ax2.set_xlabel('cos(\u03D1) [adm]')
ax2.ticklabel_format(axis="y",style="sci",scilimits=(0,1))
ax2.set_ylabel('counts')

fit_info = []
for p, v, e in zip(m2.parameters, m2.values, m2.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

ax2.legend(title="\n".join(fit_info), loc="best")

# aspect=(custom_xlim[1]/custom_ylim[1])
# ax2.set_aspect(aspect)


print("CL fitting and covarience \n", m2)

# fig.savefig(save_path+title+"_intesity_R-CR-CL", dpi=144, transparent=False, bbox_inches='tight')

In [ ]:
import functions_minuit as fumi
#PECD INTERPOLATION RANDOM
# import functions_minuit as fumi

fig, ax = plt.subplots()
custom_xlim = (-1, 1)
custom_ylim = (-0.05, 0.05)
    
# limits=[(-1, 1), (-1, 1),(-1, 1), (-1, 1),(-1, 1), (-1, 1)] #PECD6
limits=[(-1, 1), (-1, 2)] #PECD2

x_data = ctheta_all;y_data = cPECD_all_R;y_err = cPECD_all_R_err

m=fumi.iminuit_err2(x_data,y_data,y_err,limits)
fumi.plotminuit2(x_data,y_data,y_err,m,ax,custom_xlim, ylim=custom_ylim)

ax.set_title(title+" all PECD R CR-CL @ 33 eV")

ax.set_xlabel('cos(\u03D1) [adm]')
ax.set_ylabel('PECD')

fit_info = []
for p, v, e in zip(m.parameters, m.values, m.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

ax.legend(title="\n".join(fit_info), loc="best")

aspect=(custom_xlim[1]/custom_ylim[1])
ax.set_aspect(aspect)

m

fig.savefig(save_path+title+"_allPECD_R", dpi=144, transparent=False, bbox_inches='tight')

In [ ]:
import functions_minuit as fumi
#PECD INTERPOLATION RANDOM
# import functions_minuit as fumi

fig, ax = plt.subplots()
custom_xlim = (-1, 1)
custom_ylim = (-0.05, 0.05)
    
# limits=[(-1, 1), (-1, 1),(-1, 1), (-1, 1),(-1, 1), (-1, 1)] #PECD6
limits=[(-1, 1), (-1, 2)] #PECD2

x_data = ctheta_all;y_data = cPECD_all_S;y_err = cPECD_all_S_err

m=fumi.iminuit_err2(x_data,y_data,y_err,limits)
fumi.plotminuit2(x_data,y_data,y_err,m,ax,custom_xlim, ylim=custom_ylim)

ax.set_title(title+" all PECD S CR-CL @ 33 eV")

ax.set_xlabel('cos(\u03D1) [adm]')
ax.set_ylabel('PECD')

fit_info = []
for p, v, e in zip(m.parameters, m.values, m.errors):
    fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")

ax.legend(title="\n".join(fit_info), loc="best")

aspect=(custom_xlim[1]/custom_ylim[1])
ax.set_aspect(aspect)

m

fig.savefig(save_path+title+"_allPECD_S", dpi=144, transparent=False, bbox_inches='tight')

# SAMPLE SINGLE GRAPH #

fig,ax= plt.subplots(1, figsize=(12, 10))
#cs,ax=fugi.plot_interpolation(beta,ctheta,PECD_tot_R.T, xstep=0.01, ystep=0.01, ax=ax, cmap=cmap_temp, gaussian=10.)
#cs=ax.pcolormesh(beta,ctheta,PECD_tot_R.T, cmap=cmap_temp)

ax.set_title("PECD S CR-CL @ 33 eV")
aspect=(beta.max()/ctheta.max())
ax.set_xlabel('cos(\u03B2) [adm]')
ax.set_ylabel('cos(\u03D1) [adm]')
cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.ax.set_ylabel('PECD')
ax.set_aspect(aspect)

#fig.savefig(save_path+"enscan_"+PATH_name+"_"+EN[EN_opt]+"eV_norm_PECDmap_R.png", dpi=144, transparent=False)


In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2, figsize=(12, 10), constrained_layout=True, sharey=True)
aspect=(beta.max()/ctheta.max())

cs1=ax1.pcolormesh(beta,cthetaP,PECD_tot_RS.T,cmap=cmap_temp) #shading='auto'
fig.suptitle(title+" PECD R-S @ 33 eV",y=0.85)
ax1.set_title("original")
ax1.set_xlabel('cos(\u03B2) [adm]')
ax1.set_ylabel('cos(\u03D1) [adm]')
# cbar = fig.colorbar(cs1,shrink=0.9, ax=ax1)
# cbar.ax.set_ylabel('PECD')
ax1.set_aspect(aspect)

cs2,ax2=fugi.plot_interpolation(beta,cthetaP,PECD_tot_RS.T, xstep=0.01, ystep=0.01, ax=ax2, cmap=cmap_temp, gaussian=10.)
ax2.set_title("smooth")
ax2.set_xlabel('cos(\u03B2) [adm]')
# ax2.set_ylabel('cos(\u03D1) [adm]')
ax2.set_aspect(aspect)

# cax = fig.add_axes([ax2.get_position().x1+0.11,ax2.get_position().y0-0.057,0.02,ax2.get_position().height+0.108])
cbar= fig.colorbar(cs2,fraction=0.05, pad=0.04)
cbar.ax.set_ylabel('PECD')

fig.savefig(save_path+title+"_PECD_R-S", dpi=144, transparent=False, bbox_inches='tight')

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2, figsize=(12, 10), constrained_layout=True, sharey=True)
aspect=(beta.max()/ctheta.max())

cs1=ax1.pcolormesh(beta,cthetaP,PECD_tot_RS_presmooth.T,cmap=cmap_temp) #shading='auto'
fig.suptitle(title+" PECD R-S pre_sigma="+str(gaussian)+" @ 33 eV",y=0.85)
ax1.set_title("original")
ax1.set_xlabel('cos(\u03B2) [adm]')
ax1.set_ylabel('cos(\u03D1) [adm]')
# cbar = fig.colorbar(cs1,shrink=0.9, ax=ax1)
# cbar.ax.set_ylabel('PECD')
ax1.set_aspect(aspect)

cs2,ax2=fugi.plot_interpolation(beta,cthetaP,PECD_tot_RS_presmooth.T, xstep=0.01, ystep=0.01, ax=ax2, cmap=cmap_temp)
ax2.set_title("smooth")
ax2.set_xlabel('cos(\u03B2) [adm]')
# ax2.set_ylabel('cos(\u03D1) [adm]')
ax2.set_aspect(aspect)

# cax = fig.add_axes([ax2.get_position().x1+0.11,ax2.get_position().y0-0.057,0.02,ax2.get_position().height+0.108])
cbar= fig.colorbar(cs2,fraction=0.05, pad=0.04)
cbar.ax.set_ylabel('PECD')

# fig.savefig(save_path+title+"_PECD_R-S", dpi=144, transparent=False, bbox_inches='tight')

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2, figsize=(12, 10), constrained_layout=True, sharey=True)
aspect=(beta.max()/ctheta.max())

cs1=ax1.pcolormesh(beta,cthetaP,PECD_tot_R.T,cmap=cmap_temp) #shading='auto'
fig.suptitle(title+" PECD R CR-CL @ 33 eV",y=0.85)
ax1.set_title("original")
ax1.set_xlabel('cos(\u03B2) [adm]')
ax1.set_ylabel('cos(\u03D1) [adm]')
# cbar = fig.colorbar(cs1,shrink=0.9, ax=ax1)
# cbar.ax.set_ylabel('PECD')
ax1.set_aspect(aspect)

cs2,ax2=fugi.plot_interpolation(beta,cthetaP,PECD_tot_R.T, xstep=0.01, ystep=0.01, ax=ax2, cmap=cmap_temp, gaussian=10.)
ax2.set_title("smooth")
ax2.set_xlabel('cos(\u03B2) [adm]')
# ax2.set_ylabel('cos(\u03D1) [adm]')
ax2.set_aspect(aspect)

# cax = fig.add_axes([ax2.get_position().x1+0.11,ax2.get_position().y0-0.057,0.02,ax2.get_position().height+0.108])
cbar= fig.colorbar(cs2,fraction=0.05, pad=0.04)
cbar.ax.set_ylabel('PECD')

fig.savefig(save_path+title+"_PECD_R-CR-CL", dpi=144, transparent=False, bbox_inches='tight')

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2, figsize=(12, 10), constrained_layout=True, sharey=True)
aspect=(beta.max()/ctheta.max())

cs1=ax1.pcolormesh(beta,cthetaP,PECD_tot_S.T,cmap=cmap_temp) #shading='auto'
fig.suptitle(title+" PECD S CR-CL @ 33 eV",y=0.85)
ax1.set_title("original")
ax1.set_xlabel('cos(\u03B2) [adm]')
ax1.set_ylabel('cos(\u03D1) [adm]')
# cbar = fig.colorbar(cs1,shrink=0.9, ax=ax1)
# cbar.ax.set_ylabel('PECD')
ax1.set_aspect(aspect)

cs2,ax2=fugi.plot_interpolation(beta,cthetaP,PECD_tot_S.T, xstep=0.01, ystep=0.01, ax=ax2, cmap=cmap_temp, gaussian=10.)
ax2.set_title("smooth")
ax2.set_xlabel('cos(\u03B2) [adm]')
# ax2.set_ylabel('cos(\u03D1) [adm]')
ax2.set_aspect(aspect)

# cax = fig.add_axes([ax2.get_position().x1+0.11,ax2.get_position().y0-0.057,0.02,ax2.get_position().height+0.108])
cbar= fig.colorbar(cs2,fraction=0.05, pad=0.04)
cbar.ax.set_ylabel('PECD')

fig.savefig(save_path+title+"_PECD_S-CR-CL", dpi=144, transparent=False, bbox_inches='tight')

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2, figsize=(12, 10), constrained_layout=True, sharey=True)
aspect=(phiM.max()/cthetaM.max())

cs1=ax1.pcolormesh(phiM,cthetaM,MFPAD_SCL.T,shading='auto', cmap=cmap_temp)
fig.suptitle(title+" MFPAD @ 33 eV",y=0.85)
ax1.set_title("S CL")
ax1.set_xlabel('\u03C6 [DEG]')
ax1.set_ylabel('cos(\u03D1) [adm]')
# cbar = fig.colorbar(cs1,shrink=0.9, ax=ax1)
# cbar.ax.set_ylabel('PECD')
ax1.set_aspect(aspect)

cs2=ax2.pcolormesh(phiM,cthetaM,MFPAD_SCR.T,shading='auto', cmap=cmap_temp)
ax2.set_title("S CR")
ax2.set_xlabel('\u03C6 [DEG]')
# ax2.set_ylabel('cos(\u03D1) [adm]')
ax2.set_aspect(aspect)

# cax = fig.add_axes([ax2.get_position().x1+0.11,ax2.get_position().y0-0.057,0.02,ax2.get_position().height+0.108])
cbar= fig.colorbar(cs2,fraction=0.05, pad=0.04)
cbar.ax.set_ylabel('counts')

fig.savefig(save_path+title+"_singleMFPAD_S", dpi=144, transparent=False, bbox_inches='tight')

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2, figsize=(12, 10), constrained_layout=True, sharey=True)
aspect=(phiM.max()/cthetaM.max())

cs1=ax1.pcolormesh(phiM,cthetaM,MFPAD_RCL.T,shading='auto', cmap=cmap_temp)
fig.suptitle(title+" MFPAD @ 33 eV",y=0.85)
ax1.set_title("R CL")
ax1.set_xlabel('\u03C6 [DEG]')
ax1.set_ylabel('cos(\u03D1) [adm]')
# cbar = fig.colorbar(cs1,shrink=0.9, ax=ax1)
# cbar.ax.set_ylabel('PECD')
ax1.set_aspect(aspect)

cs2=ax2.pcolormesh(phiM,cthetaM,MFPAD_RCR.T,shading='auto', cmap=cmap_temp)
ax2.set_title("R CR")
ax2.set_xlabel('\u03C6 [DEG]')
# ax2.set_ylabel('cos(\u03D1) [adm]')
ax2.set_aspect(aspect)

# cax = fig.add_axes([ax2.get_position().x1+0.11,ax2.get_position().y0-0.057,0.02,ax2.get_position().height+0.108])
cbar= fig.colorbar(cs2,fraction=0.05, pad=0.04)
cbar.ax.set_ylabel('counts')

fig.savefig(save_path+title+"_singleMFPAD_R", dpi=144, transparent=False, bbox_inches='tight')

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2, figsize=(12, 10), constrained_layout=True, sharey=True)
aspect=(phiM.max()/cthetaM.max())

# cs1,ax1=fugi.plot_interpolation(phiM,cthetaM,MFPAD_R.T, xstep=0.01, ystep=0.01, ax=ax2, cmap=cmap_temp, gaussian=10.)
cs1=ax1.pcolormesh(phiM,cthetaM,MFPAD_R.T, shading='auto', cmap=cmap_temp)
fig.suptitle(title+" MFPAD @ 33 eV",y=0.85)
ax1.set_title("R")
ax1.set_xlabel('\u03C6 [DEG]')
ax1.set_ylabel('cos(\u03D1) [adm]')
# cbar = fig.colorbar(cs1,shrink=0.9, ax=ax1)
# cbar.ax.set_ylabel('PECD')
ax1.set_aspect(aspect)

# cs2,ax2=fugi.plot_interpolation(phiM,cthetaM,MFPAD_S.T, xstep=0.01, ystep=0.01, ax=ax2, cmap=cmap_temp, gaussian=10.)
cs2=ax2.pcolormesh(phiM,cthetaM,MFPAD_S.T, shading='auto', cmap=cmap_temp)
ax2.set_title("S")
ax2.set_xlabel('\u03C6 [DEG]')
# ax2.set_ylabel('cos(\u03D1) [adm]')
ax2.set_aspect(aspect)

# cax = fig.add_axes([ax2.get_position().x1+0.11,ax2.get_position().y0-0.057,0.02,ax2.get_position().height+0.108])
cbar= fig.colorbar(cs2,fraction=0.05, pad=0.04)
cbar.ax.set_ylabel('PECD')

fig.savefig(save_path+title+"_totMFPAD_R-S", dpi=144, transparent=False, bbox_inches='tight')

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2, figsize=(12, 10), constrained_layout=True, sharey=True)
aspect=(phiM.max()/cthetaM.max())

cs1=ax1.pcolormesh(phiM,cthetaM,MFPAD_tot_RS.T,cmap=cmap_temp) #shading='auto'
fig.suptitle(title+" contrast map R-S @ 33 eV",y=0.85)
ax1.set_title("original")
ax1.set_xlabel('\u03C6 [DEG]')
ax1.set_ylabel('cos(\u03D1) [adm]')
# cbar = fig.colorbar(cs1,shrink=0.9, ax=ax1)
# cbar.ax.set_ylabel('PECD')
ax1.set_aspect(aspect)

cs2,ax2=fugi.plot_interpolation(phiM,cthetaM,MFPAD_tot_RS.T, xstep=0.05, ystep=0.05, ax=ax2, cmap=cmap_temp, gaussian=10.)
ax2.set_title("smooth")
ax2.set_xlabel('\u03C6 [DEG]')
# ax2.set_ylabel('cos(\u03D1) [adm]')
ax2.set_aspect(aspect)

# cax = fig.add_axes([ax2.get_position().x1+0.11,ax2.get_position().y0-0.057,0.02,ax2.get_position().height+0.108])
cbar= fig.colorbar(cs2,fraction=0.05, pad=0.04)
cbar.ax.set_ylabel('asymmetry')

fig.savefig(save_path+title+"_contrast_map_R-S", dpi=144, transparent=False, bbox_inches='tight')